In [1]:
from Bio import Entrez
from datetime import datetime
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm as tqdm_module
import pbmd_tools as pbmd

In [2]:
Entrez.email = "nadiajuckova@gmail.com"

In [3]:
req_esearch = Entrez.esearch(db = "pubmed", term = "github", retmax = 15000)
res_esearch = Entrez.read(req_esearch)
req_esummary = Entrez.esummary(db = "pubmed", id = ",".join(res_esearch["IdList"]))
res_esummary = Entrez.read(req_esummary)
len(res_esummary)

IncompleteRead: IncompleteRead(1321 bytes read)

In [ ]:
df = pd.DataFrame.from_records(res_esummary)
df = df[["Id", "DOI", "History", "FullJournalName", "Title"]]
df["PubDate"] = df["History"].apply(lambda x: x["entrez"] if "entrez" in x else None)
df["PubDate"] = df["PubDate"].apply(lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M").strftime("%Y-%m-%d") if x else None)
df = df[["Id", "DOI", "PubDate", "FullJournalName", "Title"]]
df

In [23]:
for id_iter in df["Id"]:  
    df.loc[df["Id"] == id_iter,"Abstract"] = pbmd.get_abstract(id_iter)

AttributeError: module 'pbmd_tools' has no attribute 'get_abstract'

In [164]:
df[df['Abstract'].isna()]

,Id,DOI,PubDate,GitHub_link_raw,FullJournalName,Title,Phrase,Abstract
1681,35811294,10.1016/S2589-7500(22)00127-3,2022-07-10,,The Lancet. Digital health,Early human judgment forecasts of human monkey...,,NaN
2522,35337638,10.1016/S2589-7500(22)00032-2,2022-03-26,,The Lancet. Digital health,"An interactive dashboard to track themes, deve...",,NaN
3669,34627498,10.1016/S1473-3099(21)00620-4,2021-10-10,,The Lancet. Infectious diseases,Implications of suboptimal COVID-19 vaccinatio...,,NaN
4379,34197809,10.1016/S0140-6736(21)01462-8,2021-07-01,,"Lancet (London, England)",AZD1222-induced neutralising antibody activity...,,NaN
4564,34090624,10.1016/S0140-6736(21)01290-3,2021-06-06,,"Lancet (London, England)",Neutralising antibody activity against SARS-Co...,,NaN
8479,33937797,10.1148/ryai.2019190072,2021-05-03,,Radiology. Artificial intelligence,Magician's Corner: How to Start Learning about...,,NaN
8576,31281945,10.1186/s13321-019-0365-4,2019-07-09,,Journal of cheminformatics,"Journal of Cheminformatics, ORCID, and GitHub.",,NaN


In [19]:
for id_iter in df["Id"]:
    
    df.loc[df["Id"] == id_iter,"GitHub_link_raw"] = ", ".join(pbmd.get_link_github(df, id_iter))
    df.loc[df["Id"] == id_iter,"GitHub_link_clean"] = ", ".join(pbmd.clean_link(df.loc[df["Id"] == id_iter,"GitHub_link_raw"]))
    df.loc[df["Id"] == id_iter, "Phrase"] = str(pbmd.get_phrase_with_link(df, id_iter))

df = df[["Id", "DOI", "PubDate", "GitHub_link_raw", "GitHub_link_clean", "FullJournalName", "Title", "Phrase", "Abstract"]]

AttributeError: module 'pbmd_tools' has no attribute 'get_link_github'

In [14]:
pd.options.display.max_colwidth = None
df[["Id", "DOI", "PubDate", "GitHub_link_raw", "GitHub_link_clean", "FullJournalName", "Title", "Phrase"]].head(100)

,Id,DOI,PubDate,GitHub_link_raw,GitHub_link_clean,FullJournalName,Title,Phrase
0,36732776,10.1186/s13040-023-00321-5,2023-02-03,https://github.com/CirculatoryHealth/LoFTK,https://github.com/CirculatoryHealth/LoFTK,BioData mining,LoFTK: a framework for fully automated calculation of predicted Loss-of-Function variants and genes.,LoFTK is an open software and is freely available to non-commercial users at https://github.com/CirculatoryHealth/LoFTK .
1,36732704,10.1186/s12859-023-05158-7,2023-02-02,https://github.com/tyqGitHub/TYQ/tree/master/GACNNMDA,https://github.com/tyqGitHub/TYQ/tree/master/GACNNMDA,BMC bioinformatics,GACNNMDA: a computational model for predicting potential human microbe-drug associations based on graph attention network and CNN-based classifier.,Source codes and supplementary materials are available at: https://github.com/tyqGitHub/TYQ/tree/master/GACNNMDA.
2,36732214,10.1121/10.0016850,2023-02-02,https://ucrbraingamecenter.github.io/PART_Utilities/,https://ucrbraingamecenter.github.io/PART_Utilities/,The Journal of the Acoustical Society of America,Development and validation of a Spanish-language spatial release from masking task in a Mexican population.,"To present the Spanish-language SRM test, we created new recordings of the CRM with Spanish-language Translations and updated the freely available app (PART; https://ucrbraingamecenter.github.io/PART_Utilities/) to present materials in Spanish."
3,36731618,10.1016/j.gene.2023.147231,2023-02-02,https://github.com/almeidasilvaf/SoyPestGCN,https://github.com/almeidasilvaf/SoyPestGCN,Gene,Discovering and prioritizing candidate resistance genes against soybean pests by integrating GWAS and gene coexpression networks.,r/) and an R/Shiny package (https://github.com/almeidasilvaf/SoyPestGCN) that serve as a public resource to explore soybean-pest interactions at the transcriptional level.
4,36731360,10.1016/j.compbiomed.2023.106585,2023-02-02,https://github.com/CloneRob/ECMT,https://github.com/CloneRob/ECMT,Computers in biology and medicine,Error-Correcting Mean-Teacher: Corrections instead of consistency-targets applied to semi-supervised medical image segmentation.,Code is available at https://github.com/CloneRob/ECMT.
...,...,...,...,...,...,...,...,...
95,36699385,10.1093/bioadv/vbac016,2023-01-26,"https://github.com/saezlab/decoupler-py, https://github.com/saezlab/decoupleR_manuscript","https://github.com/saezlab/decoupler-py, https://github.com/saezlab/decoupleR_manuscript",Bioinformatics advances,decoupleR: ensemble of computational methods to infer biological activities from omics data.,tml) for R and in GitHub (https://github.com/saezlab/decoupler-py) for Python.
96,36699384,10.1093/bioadv/vbac005,2023-01-26,"https://github.com/AdamaJava/adamajava,",https://github.com/AdamaJava/adamajava,Bioinformatics advances,qmotif: determination of telomere content from whole-genome sequence data.,"qmotif is implemented in Java and source code is available at https://github.com/AdamaJava/adamajava, with instructions on how to build and use the application available from https://adamajava.readthedocs.io/en/latest/."
97,36699383,10.1093/bioadv/vbac025,2023-01-26,https://github.com/jchow32/MAGI-MS,https://github.com/jchow32/MAGI-MS,Bioinformatics advances,MAGI-MS: multiple seed-centric module discovery.,MAGI-MS is free and available at https://github.com/jchow32/MAGI-MS. Supplementary data are available at <i>Bioinformatics Advances</i> online.
98,36699382,10.1093/bioadv/vbac056,2023-01-26,github.com/pipolinlab/ExplorePipolin,https://github.com/pipolinlab/ExplorePipolin,Bioinformatics advances,ExplorePipolin: reconstruction and annotation of piPolB-encoding bacterial mobile elements from draft genomes.,ll code is available and can be accessed here: github.com/pipolinlab/ExplorePipolin.


In [15]:
df[["Id", "DOI", "PubDate", "GitHub_link_raw", "GitHub_link_clean", "FullJournalName", "Title", "Phrase"]][(df['GitHub_link_raw'] == '') & (df['Phrase'] == '')]

,Id,DOI,PubDate,GitHub_link_raw,GitHub_link_clean,FullJournalName,Title,Phrase


In [16]:
df[["Id", "DOI", "PubDate", "GitHub_link_raw", "GitHub_link_clean", "FullJournalName", "Title", "Phrase"]][df['Phrase'] == '']

,Id,DOI,PubDate,GitHub_link_raw,GitHub_link_clean,FullJournalName,Title,Phrase


In [17]:
df[["Id", "DOI", "PubDate", "GitHub_link_raw", "GitHub_link_clean", "FullJournalName", "Title", "Phrase"]][df['GitHub_link_raw'] == '']

,Id,DOI,PubDate,GitHub_link_raw,GitHub_link_clean,FullJournalName,Title,Phrase


In [270]:
df.to_csv('C:/Users/nadia/pubmed_results_10000.tsv', sep='\t', index=False, columns=df.columns[:-1].tolist())